In [1]:
print ("hello world")

hello world


In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\chat-bot\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS\\Desktop\\chat-bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\ASUS\anaconda3\envs\chatbot_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# --- IMPORTS ---
# We use PyMuPDFLoader because it reads tables and layout better
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader

# --- FUNCTION DEFINITION ---
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader  # This forces it to use the better loader
    )
    
    documents = loader.load()
    return documents




In [7]:

folder_path = r"C:\Users\ASUS\Desktop\chat-bot\data" 
extracted_data = load_pdf_files(folder_path)
print(f"Successfully loaded {len(extracted_data)} pages.")

Successfully loaded 421 pages.


In [8]:
extracted_data

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-10-03T04:26:32+00:00', 'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf', 'file_path': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf', 'total_pages': 76, 'format': 'PDF 1.5', 'title': 'Guidelins for Elimination of Mother to Child Transmission of HIV', 'author': 'Ministry of Health-NSACP', 'subject': '', 'keywords': '', 'moddate': '2024-10-03T04:26:32+00:00', 'trapped': '', 'modDate': 'D:20241003042632Z', 'creationDate': "D:20241003042632+00'00'", 'page': 0}, page_content='1 \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGuidelines for Management of Pregnant Women \n with HIV Infection  2024 \n \nNational STD/AIDS Control Programme \nMinistry of Health \nSri Lanka \

In [9]:
len(extracted_data)

421

In [15]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Returns docs with only 'source', 'page', and 'page_content'.
    """
    minimal_docs: List[Document] = []
    
    for doc in docs:
        # Extract the specific metadata you need
        src = doc.metadata.get("source")
        page_num = doc.metadata.get("page", 0) # Default to 0 if no page found
        
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={
                    "source": src,
                    "page": page_num + 1  # PyMuPDF starts at 0, humans start at 1
                }
            )
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
minimal_docs

[Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf'}, page_content='1 \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGuidelines for Management of Pregnant Women \n with HIV Infection  2024 \n \nNational STD/AIDS Control Programme \nMinistry of Health \nSri Lanka \n \n                 \n \nNational Programme on Elimination of Mother to Child Transmission \n of HIV and Syphilis in Sri Lanka'),
 Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf'}, page_content='2 \n \n \nGuidelines for Management of Pregnant Women \nwith HIV Infection  2024 \n \n \n \n \n \n \n \n \n \n \nNational STD/AIDS Control Programme \nMinistry of Health  Sri Lanka'),
 Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_mana

In [17]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,      # Big enough for tables, fits in MiniLM limit
        chunk_overlap=100,   # Ensures sentences/logic aren't cut off
        length_function=len,
        separators=["\n\n", "\n", " ", ""] # Tries to split by paragraph first
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [18]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1110


In [19]:
texts_chunk

[Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf'}, page_content='1 \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGuidelines for Management of Pregnant Women \n with HIV Infection  2024 \n \nNational STD/AIDS Control Programme \nMinistry of Health \nSri Lanka \n \n                 \n \nNational Programme on Elimination of Mother to Child Transmission \n of HIV and Syphilis in Sri Lanka'),
 Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_management_of_pregnant_women_with_HIV_infection_2024.pdf'}, page_content='2 \n \n \nGuidelines for Management of Pregnant Women \nwith HIV Infection  2024 \n \n \n \n \n \n \n \n \n \n \nNational STD/AIDS Control Programme \nMinistry of Health  Sri Lanka'),
 Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\Guidelines_for_mana

In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    # Using MPNET because it allows 2000+ characters per chunk
    # This is perfect for your 1100 chunk size!
    model_name = "sentence-transformers/all-mpnet-base-v2"
    
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25788\3442255882.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\ASUS\anaconda3\envs\chatbot_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` en

In [21]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
vector = embedding.embed_query("Hello world")
vector

[0.02624971978366375,
 0.013395597226917744,
 -0.004533165134489536,
 -0.021791458129882812,
 0.05455183982849121,
 -0.004966510459780693,
 0.006655589677393436,
 0.030626295134425163,
 -0.005762827582657337,
 -0.004562050104141235,
 -0.003313298337161541,
 -0.04849625006318092,
 -0.011364003643393517,
 0.03507741168141365,
 0.09309464693069458,
 -0.08668739348649979,
 0.051086556166410446,
 0.009886142797768116,
 -0.06356924772262573,
 -0.0085501903668046,
 0.007054393645375967,
 -0.003862416837364435,
 0.0247443076223135,
 0.04288493096828461,
 0.035094134509563446,
 -0.029848245903849602,
 0.010252577252686024,
 0.022344952449202538,
 0.02088995650410652,
 0.009492229670286179,
 -0.03304444998502731,
 -0.01228412240743637,
 0.053528930991888046,
 0.025429194793105125,
 2.022176886384841e-06,
 -0.034191031008958817,
 0.009609980508685112,
 -0.01648453250527382,
 0.005609510000795126,
 -0.004250049591064453,
 -0.02280120551586151,
 0.040354691445827484,
 0.0030520367436110973,
 0.0313

In [23]:
print( "Vector length:", len(vector))

Vector length: 768


In [24]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [26]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY



In [27]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [28]:
pc

In [29]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=768,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [32]:
from langchain_pinecone import PineconeVectorStore
import time

print("Starting manual upload...")

# 1. Initialize the Vector Store (But don't add documents yet)
docsearch = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding
)

# 2. Define a small batch size (Safe for your internet)
batch_limit = 5 

# 3. Loop through your chunks and add them piece by piece
total_chunks = len(texts_chunk)

for i in range(0, total_chunks, batch_limit):
    # Slice the list to get just 5 documents
    batch = texts_chunk[i : i + batch_limit]
    
    # Upload them
    docsearch.add_documents(batch)
    
    # Print progress
    print(f"Uploaded: {min(i + batch_limit, total_chunks)} / {total_chunks}")
    
    # Small sleep to prevent hitting API rate limits
    time.sleep(1)

print("Upload Complete!")

Starting manual upload...
Uploaded: 5 / 1110
Uploaded: 10 / 1110
Uploaded: 15 / 1110
Uploaded: 20 / 1110
Uploaded: 25 / 1110
Uploaded: 30 / 1110
Uploaded: 35 / 1110
Uploaded: 40 / 1110
Uploaded: 45 / 1110
Uploaded: 50 / 1110
Uploaded: 55 / 1110
Uploaded: 60 / 1110
Uploaded: 65 / 1110
Uploaded: 70 / 1110
Uploaded: 75 / 1110
Uploaded: 80 / 1110
Uploaded: 85 / 1110
Uploaded: 90 / 1110
Uploaded: 95 / 1110
Uploaded: 100 / 1110
Uploaded: 105 / 1110
Uploaded: 110 / 1110
Uploaded: 115 / 1110
Uploaded: 120 / 1110
Uploaded: 125 / 1110
Uploaded: 130 / 1110
Uploaded: 135 / 1110
Uploaded: 140 / 1110
Uploaded: 145 / 1110
Uploaded: 150 / 1110
Uploaded: 155 / 1110
Uploaded: 160 / 1110
Uploaded: 165 / 1110
Uploaded: 170 / 1110
Uploaded: 175 / 1110
Uploaded: 180 / 1110
Uploaded: 185 / 1110
Uploaded: 190 / 1110
Uploaded: 195 / 1110
Uploaded: 200 / 1110
Uploaded: 205 / 1110
Uploaded: 210 / 1110
Uploaded: 215 / 1110
Uploaded: 220 / 1110
Uploaded: 225 / 1110
Uploaded: 230 / 1110
Uploaded: 235 / 1110
Uploade

In [33]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [34]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [35]:
retrieved_docs = retriever.invoke("How does tuberculosis spread?")
retrieved_docs

[Document(id='7974a5a9-ca8b-48a7-976b-883253798ed4', metadata={'source': 'C:\\Users\\ASUS\\Desktop\\chat-bot\\data\\maternal_care_package_a_guide_to_field_healthcare_workers_english.pdf'}, page_content='Maternal Care Package A Guide to Field Healthcare Workers \n147 \n \n \nChapter 21  \n \n \n \nMANAGEMENT OF TUBERCULOSIS DURING PREGNANCY \n \nTuberculosis is an infectious disease caused by the bacillus- Mycobacterium tuberculosis and \noccasionally by Mycobacterium bovis and Mycobacterium africanum. Tuberculosis commonly affects \nthe lungs, but it can affect any other organ in the body. \n \nHow does tuberculosis spread?  \nThe bacteria that cause tuberculosis usually spread through air. When a patient with infectious \npulmonary tuberculosis coughs, sneezes or laughs, bacilli are expelled into the air in the form of tiny \ndroplets. These droplets dry up rapidly to form droplet nuclei and may remain suspended in the air for'),
 Document(id='334dc4e5-170d-4a0c-b88e-fc863680a804', me

In [36]:
# 1. Import the Google GenAI library
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI


c:\Users\ASUS\anaconda3\envs\chatbot_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [37]:
load_dotenv()

True

In [43]:
chatModel = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3  # Low temperature = More factual answers (Good for Medical)
)

In [44]:
# These are universal imports. Keep them exactly like this.
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [50]:
system_prompt = (
    "You are a Medical Assistant specialized in Clinical Guidelines. "
    "Use the following pieces of retrieved context to answer the question. "
    
    "RULES:"
    "1. If the user asks for treatments, you MUST list the specific drug names, dosages, and regimens found in the context."
    "2. Do NOT summarize medical protocols. Be exact."
    "3. If the answer is in a table in the context, try to present it clearly."
    "4. If the user asks in Sinhala, answer in Sinhala."
    "5. If you don't know, say you don't know."
    
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Re-create the chain with the new prompt
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
response = rag_chain.invoke({"input": "What are the treatments for HIV in pregnancy?"})
print(response["answer"])

All pregnant and breastfeeding women living with HIV should initiate and continue lifelong ART, regardless of WHO clinical stage or CD4 count. Pregnant women newly diagnosed with HIV should start ART as early as possible, while those already on ART with sustained viral suppression should maintain their current regimen.



In [42]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# 1. Load your API Key
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

# 2. Configure the Google Library
genai.configure(api_key=api_key)

print("--- Available Chat Models ---")

# 3. Ask Google for the list
try:
    for m in genai.list_models():
        # We only want models that can generate text (Chat)
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print("Error:", e)

--- Available Chat Models ---
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flas

In [47]:
response = rag_chain.invoke({"input": "hi what can you do?"})
print(response["answer"])

I can answer questions based on the provided medical context. I can provide information related to maternal and child health, family planning, reproductive health, and general health advice. I can also give advice on newborn care, nutrition, and danger signs during pregnancy.



In [48]:
response = rag_chain.invoke({"input": "How does tuberculosis spread?"})
print(response["answer"])

Tuberculosis spreads through the air when a person with infectious pulmonary tuberculosis coughs, sneezes, or laughs, releasing bacilli into the air in tiny droplets. A healthy person can become infected by inhaling these droplets. Ventilation and direct sunlight can help to remove and kill the bacilli.



In [52]:
response = rag_chain.invoke({"input": "Hi"})
print(response["answer"])

Hi there! How can I help you today?



In [53]:
response = rag_chain.invoke({"input": "how can you help me?"})
print(response["answer"])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..


I can help you by:

*   Enquiring about family support.
*   Solving problems in sexual intercourse.
*   Inquiring about the mother’s mental status and supporting the mother to develop her self-esteem.
*   Giving advice on whom to call or where to go in case of bleeding, abdominal pain, and any other emergency, or when in need of other advice.
*   Emphasizing the need for continuation of treatment for medical conditions.
*   Giving advice on how to contact PHM in an emergency (mobile numbers, place of the office and residence).
*   Requesting the woman to record when she notes the first fetal movement.
*   Discussing with the woman and the family regarding the importance of maintaining good mental status during pregnancy.
*   Discussing with the woman and the husband (or a family member) on the importance of attending antenatal clinics and antenatal classes so that they can be involved in the activities and can learn how to support the woman through her pregnancy.


In [54]:
response = rag_chain.invoke({"input": "can you give me how to contact PHM in an emergency?"})
print(response["answer"])

Contact the PHM in an emergency using the following methods:

*   Mobile numbers
*   Place of the office
*   Residence


In [56]:
response = rag_chain.invoke({"input": " what are the screening for postpartum depression and general advice for postpartum care.?"})
print(response["answer"])

Screening for Postpartum Depression:
*   Edinburgh Postnatal Depression Scale (EPDS): This consists of ten questions and can usually be completed in less than 5 minutes. Responses are scored 0, 1, 2, or 3 according to increased severity of the symptom. The total score is determined by adding together the scores for each of the 10 items.

General advice for postpartum care:

*   Assess the mental condition of the mother – rest, sleep, unhappiness, loneliness
*   All women should be asked about resolution of symptoms of maternal blues. If symptoms have not resolved, the woman’s psychological well-being should continue to be assessed for postnatal depression. Refer to MOH clinic.
*   Continue to observe for any indication of domestic abuse.
*   Complete all the relevant records.
*   Mothers with problems may need more visits. Decide on date of next visit according to the problems identified.

